# 네이버 140자평 데이터 수집

상영장 예매순위 top 15 영화에 대한 140자평 데이터를 수집한다.

가져오고자 하는 요소가 1개가 아니기 때문에 각각 요소를 구분할 수 있는 값을 먼저 추출하고 그 값을 이용해 각각의 영화를 모두 요청해서 데이터를 가져온다.

In [169]:
import bs4
import requests
import time
import pandas
import os

In [2]:
# 접속할 페이지의 주소
# 예매순 첫번째 영화의 주소 (나중에 바꿔주기!)
site = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=176306'

In [21]:
# 요청
response = requests.get(site)
response

<Response [200]>

In [6]:
# bs4 객체를 생성
soup = bs4.BeautifulSoup(response.text,'lxml')
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no" http-equiv="imagetoolbar"/>
<title>남산의 부장들 : 네이버 영화</title>
<meta content="https://ssl.pstatic.net/imgmovie/today/naverme/naverme_profile.jpg" property="me2:image"/>
<meta content="네이버영화 남산의 부장들" property="me2:post_tag"/>
<meta content="네이버영화" property="me2:category1"/>
<meta content="남산의 부장들" property="me2:category2"/>
<meta content="남산의 부장들" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="https://movie.naver.com/movie/bi/mi/basic.nhn?code=176306" property="og:url"/>
<meta content="https://movie-phinf.pstatic.net/20200103_41/1578032244761ejKMX_JPEG/movie_image.jpg?type=m665_443_2" property="og:image"/>
<meta content="1979년 10월 26일, 중앙정보부장 김규평(이병헌)이 대한민국 대통령을 암살한다.이 사건의 40일전, 미국..." property="og:description"/>
<meta content="https://ssl.pstatic.net/imgmovie/today/naverme/naverme_profile.jpg" property="og:article:thum

In [10]:
# 상단에 영화 목록을 가지고 있는 ul 태그를 가져온다.
ul_tag = soup.find('ul', class_ = 'top_thumb_lst')
ul_tag

<ul class="top_thumb_lst">
<li class="on" data-title="남산의 부장들"><a href="/movie/bi/mi/basic.nhn?code=176306"><img alt="남산의 부장들" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200103_41/1578032244761ejKMX_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a></li>
<li data-title="히트맨"><a href="/movie/bi/mi/basic.nhn?code=185838"><img alt="히트맨" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_181/15796574636309Fux2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a></li>
<li data-title="미스터 주: 사라진 VIP"><a href="/movie/bi/mi/basic.nhn?code=177509"><img alt="미스터 주: 사라진 VIP" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_11/15796766202282GrU2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a></li>
<li data-title="스파이 지니어스"><a href=

In [11]:
# a 태그들을 가져온다.
a_list = ul_tag.find_all('a')
a_list

[<a href="/movie/bi/mi/basic.nhn?code=176306"><img alt="남산의 부장들" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200103_41/1578032244761ejKMX_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a>,
 <a href="/movie/bi/mi/basic.nhn?code=185838"><img alt="히트맨" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_181/15796574636309Fux2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a>,
 <a href="/movie/bi/mi/basic.nhn?code=177509"><img alt="미스터 주: 사라진 VIP" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_11/15796766202282GrU2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a>,
 <a href="/movie/bi/mi/basic.nhn?code=172780"><img alt="스파이 지니어스" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" sr

In [18]:
# a 태크의 href 속성 값을 가져온다.
code_list = []

for a_tag in a_list:
    href = a_tag.attrs['href']
    str1 = href.split('=')
    code = str1[1]
#     print(code)
    code_list.append(code)
# print(code_list)

['176306', '185838', '177509', '172780', '180025', '159866', '185282', '76960', '191431', '190336', '187940', '144694', '136873', '184571', '185357']


#### 영화 140자 평 데이터를 추출

In [22]:
# 영화 코드를 받아 응답 결과를 반환하는 함수
def getHTML(code):
    site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    # 요청한다.
    response = requests.get(site)
    return response.text

In [53]:
# 전체 페이지 수를 반환하는 함수
def getPageCnt(html):
    # bs4 객체 생성
    soup = bs4.BeautifulSoup(html,'lxml')
    # 전체 글 개수를 가져온다.
    div_tag = soup.find('div', class_='score_total')
    em_tag = div_tag.find('em')
    str_total_cnt = em_tag.text.replace(',','')
    str_total_cnt = int(str_total_cnt)
    
    # 페이지 수를 계산한다.
    page_cnt = str_total_cnt // 10
    if str_total_cnt % 10 >0:
        page_cnt = page_cnt + 1
    return page_cnt

In [188]:
# 현재 페이지에서 140자 평 데이터를 가져온다.
def get140comment(code, page):
    site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
    # 요청한다.
    response = requests.get(site)
    # bs4 객체를 생성
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    # 데이터를 가지고 있는 div 태그를 가져온다.
    div_tag = soup.find('div', class_='score_result')
    # 내부에 있는 li 태그들을 가져온다.
    li_list = div_tag.find_all('li')
    
    for li_tag in li_list:
        # 별점을 가져온다.
        star_score_tag = li_tag.find('div',class_='star_score')
        em_tag = star_score_tag.find('em')
        star_score = em_tag.text # 별점
        
        # 평점글을 가져온다.
        score_reple_tag = li_tag.find('div',class_='score_reple')
        span_list = score_reple_tag.find_all('span')
        if len(span_list) > 2:   # 관람객 평점
            score_reple = span_list[1].text.strip()
            writer_id = span_list[2].text.strip()
        else:                    # 일반
            score_reple = span_list[0].text.strip()
            writer_id = span_list[1].text.strip()
        
        # 작성 날짜
        dl_tag = li_tag.find('dl')
        em_list = dl_tag.find_all('em')
        write_date = em_list[1].text
        
        reple_tuple = star_score, score_reple, writer_id, write_date
        
        # 수집한 데이터를 저장한다.
        
        if os.path.isfile(f'movie_data_{code}.csv') == False:
            columns = ['star_score', 'score_reple', 'writer_id', 'write_date']
            df1 = pandas.DataFrame([reple_tuple], columns=columns)
            df1.to_csv(f'movie_data_{code}.csv', index=False, encoding='utf-8-sig')
        else:
            df1 = pandas.DataFrame([reple_tuple])
            df1.to_csv(f'movie_data_{code}.csv', mode='a', index=False, header=False, encoding='utf-8-sig')
        

In [189]:
code_list=['156464']
for movie_code in code_list:
    html = getHTML(movie_code)
    page_cnt = getPageCnt(html)

    # 페이지 수만큼 반복
    for idx in range(1, page_cnt+1):
        time.sleep(0.005)
        print(f'code : {movie_code}, page : {idx} / {page_cnt}')
        get140comment(movie_code, idx)
print('수집 완료')

code : 156464, page : 1 / 3890
code : 156464, page : 2 / 3890
code : 156464, page : 3 / 3890
code : 156464, page : 4 / 3890
code : 156464, page : 5 / 3890
code : 156464, page : 6 / 3890
code : 156464, page : 7 / 3890
code : 156464, page : 8 / 3890
code : 156464, page : 9 / 3890
code : 156464, page : 10 / 3890
code : 156464, page : 11 / 3890
code : 156464, page : 12 / 3890
code : 156464, page : 13 / 3890
code : 156464, page : 14 / 3890
code : 156464, page : 15 / 3890
code : 156464, page : 16 / 3890
code : 156464, page : 17 / 3890
code : 156464, page : 18 / 3890
code : 156464, page : 19 / 3890
code : 156464, page : 20 / 3890
code : 156464, page : 21 / 3890
code : 156464, page : 22 / 3890
code : 156464, page : 23 / 3890
code : 156464, page : 24 / 3890
code : 156464, page : 25 / 3890
code : 156464, page : 26 / 3890
code : 156464, page : 27 / 3890
code : 156464, page : 28 / 3890
code : 156464, page : 29 / 3890
code : 156464, page : 30 / 3890
code : 156464, page : 31 / 3890
code : 156464, pa

code : 156464, page : 254 / 3890
code : 156464, page : 255 / 3890
code : 156464, page : 256 / 3890
code : 156464, page : 257 / 3890
code : 156464, page : 258 / 3890
code : 156464, page : 259 / 3890
code : 156464, page : 260 / 3890
code : 156464, page : 261 / 3890
code : 156464, page : 262 / 3890
code : 156464, page : 263 / 3890
code : 156464, page : 264 / 3890
code : 156464, page : 265 / 3890
code : 156464, page : 266 / 3890
code : 156464, page : 267 / 3890
code : 156464, page : 268 / 3890
code : 156464, page : 269 / 3890
code : 156464, page : 270 / 3890
code : 156464, page : 271 / 3890
code : 156464, page : 272 / 3890
code : 156464, page : 273 / 3890
code : 156464, page : 274 / 3890
code : 156464, page : 275 / 3890
code : 156464, page : 276 / 3890
code : 156464, page : 277 / 3890
code : 156464, page : 278 / 3890
code : 156464, page : 279 / 3890
code : 156464, page : 280 / 3890
code : 156464, page : 281 / 3890
code : 156464, page : 282 / 3890
code : 156464, page : 283 / 3890
code : 156

code : 156464, page : 504 / 3890
code : 156464, page : 505 / 3890
code : 156464, page : 506 / 3890
code : 156464, page : 507 / 3890
code : 156464, page : 508 / 3890
code : 156464, page : 509 / 3890
code : 156464, page : 510 / 3890
code : 156464, page : 511 / 3890
code : 156464, page : 512 / 3890
code : 156464, page : 513 / 3890
code : 156464, page : 514 / 3890
code : 156464, page : 515 / 3890
code : 156464, page : 516 / 3890
code : 156464, page : 517 / 3890
code : 156464, page : 518 / 3890
code : 156464, page : 519 / 3890
code : 156464, page : 520 / 3890
code : 156464, page : 521 / 3890
code : 156464, page : 522 / 3890
code : 156464, page : 523 / 3890
code : 156464, page : 524 / 3890
code : 156464, page : 525 / 3890
code : 156464, page : 526 / 3890
code : 156464, page : 527 / 3890
code : 156464, page : 528 / 3890
code : 156464, page : 529 / 3890
code : 156464, page : 530 / 3890
code : 156464, page : 531 / 3890
code : 156464, page : 532 / 3890
code : 156464, page : 533 / 3890
code : 156

code : 156464, page : 753 / 3890
code : 156464, page : 754 / 3890
code : 156464, page : 755 / 3890
code : 156464, page : 756 / 3890
code : 156464, page : 757 / 3890
code : 156464, page : 758 / 3890
code : 156464, page : 759 / 3890
code : 156464, page : 760 / 3890
code : 156464, page : 761 / 3890
code : 156464, page : 762 / 3890
code : 156464, page : 763 / 3890
code : 156464, page : 764 / 3890
code : 156464, page : 765 / 3890
code : 156464, page : 766 / 3890
code : 156464, page : 767 / 3890
code : 156464, page : 768 / 3890
code : 156464, page : 769 / 3890
code : 156464, page : 770 / 3890
code : 156464, page : 771 / 3890
code : 156464, page : 772 / 3890
code : 156464, page : 773 / 3890
code : 156464, page : 774 / 3890
code : 156464, page : 775 / 3890
code : 156464, page : 776 / 3890
code : 156464, page : 777 / 3890
code : 156464, page : 778 / 3890
code : 156464, page : 779 / 3890
code : 156464, page : 780 / 3890
code : 156464, page : 781 / 3890
code : 156464, page : 782 / 3890
code : 156

code : 156464, page : 1002 / 3890
code : 156464, page : 1003 / 3890
code : 156464, page : 1004 / 3890
code : 156464, page : 1005 / 3890
code : 156464, page : 1006 / 3890
code : 156464, page : 1007 / 3890
code : 156464, page : 1008 / 3890
code : 156464, page : 1009 / 3890
code : 156464, page : 1010 / 3890
code : 156464, page : 1011 / 3890
code : 156464, page : 1012 / 3890
code : 156464, page : 1013 / 3890
code : 156464, page : 1014 / 3890
code : 156464, page : 1015 / 3890
code : 156464, page : 1016 / 3890
code : 156464, page : 1017 / 3890
code : 156464, page : 1018 / 3890
code : 156464, page : 1019 / 3890
code : 156464, page : 1020 / 3890
code : 156464, page : 1021 / 3890
code : 156464, page : 1022 / 3890
code : 156464, page : 1023 / 3890
code : 156464, page : 1024 / 3890
code : 156464, page : 1025 / 3890
code : 156464, page : 1026 / 3890
code : 156464, page : 1027 / 3890
code : 156464, page : 1028 / 3890
code : 156464, page : 1029 / 3890
code : 156464, page : 1030 / 3890
code : 156464,

...

code : 156464, page : 2690 / 3890
code : 156464, page : 2691 / 3890
code : 156464, page : 2692 / 3890
code : 156464, page : 2693 / 3890
code : 156464, page : 2694 / 3890
code : 156464, page : 2695 / 3890
code : 156464, page : 2696 / 3890
code : 156464, page : 2697 / 3890
code : 156464, page : 2698 / 3890
code : 156464, page : 2699 / 3890
code : 156464, page : 2700 / 3890
code : 156464, page : 2701 / 3890
code : 156464, page : 2702 / 3890
code : 156464, page : 2703 / 3890
code : 156464, page : 2704 / 3890
code : 156464, page : 2705 / 3890
code : 156464, page : 2706 / 3890
code : 156464, page : 2707 / 3890
code : 156464, page : 2708 / 3890
code : 156464, page : 2709 / 3890
code : 156464, page : 2710 / 3890
code : 156464, page : 2711 / 3890
code : 156464, page : 2712 / 3890
code : 156464, page : 2713 / 3890
code : 156464, page : 2714 / 3890
code : 156464, page : 2715 / 3890
code : 156464, page : 2716 / 3890
code : 156464, page : 2717 / 3890
code : 156464, page : 2718 / 3890
code : 156464,

code : 156464, page : 2931 / 3890
code : 156464, page : 2932 / 3890
code : 156464, page : 2933 / 3890
code : 156464, page : 2934 / 3890
code : 156464, page : 2935 / 3890
code : 156464, page : 2936 / 3890
code : 156464, page : 2937 / 3890
code : 156464, page : 2938 / 3890
code : 156464, page : 2939 / 3890
code : 156464, page : 2940 / 3890
code : 156464, page : 2941 / 3890
code : 156464, page : 2942 / 3890
code : 156464, page : 2943 / 3890
code : 156464, page : 2944 / 3890
code : 156464, page : 2945 / 3890
code : 156464, page : 2946 / 3890
code : 156464, page : 2947 / 3890
code : 156464, page : 2948 / 3890
code : 156464, page : 2949 / 3890
code : 156464, page : 2950 / 3890
code : 156464, page : 2951 / 3890
code : 156464, page : 2952 / 3890
code : 156464, page : 2953 / 3890
code : 156464, page : 2954 / 3890
code : 156464, page : 2955 / 3890
code : 156464, page : 2956 / 3890
code : 156464, page : 2957 / 3890
code : 156464, page : 2958 / 3890
code : 156464, page : 2959 / 3890
code : 156464,

code : 156464, page : 3172 / 3890
code : 156464, page : 3173 / 3890
code : 156464, page : 3174 / 3890
code : 156464, page : 3175 / 3890
code : 156464, page : 3176 / 3890
code : 156464, page : 3177 / 3890
code : 156464, page : 3178 / 3890
code : 156464, page : 3179 / 3890
code : 156464, page : 3180 / 3890
code : 156464, page : 3181 / 3890
code : 156464, page : 3182 / 3890
code : 156464, page : 3183 / 3890
code : 156464, page : 3184 / 3890
code : 156464, page : 3185 / 3890
code : 156464, page : 3186 / 3890
code : 156464, page : 3187 / 3890
code : 156464, page : 3188 / 3890
code : 156464, page : 3189 / 3890
code : 156464, page : 3190 / 3890
code : 156464, page : 3191 / 3890
code : 156464, page : 3192 / 3890
code : 156464, page : 3193 / 3890
code : 156464, page : 3194 / 3890
code : 156464, page : 3195 / 3890
code : 156464, page : 3196 / 3890
code : 156464, page : 3197 / 3890
code : 156464, page : 3198 / 3890
code : 156464, page : 3199 / 3890
code : 156464, page : 3200 / 3890
code : 156464,

code : 156464, page : 3413 / 3890
code : 156464, page : 3414 / 3890
code : 156464, page : 3415 / 3890
code : 156464, page : 3416 / 3890
code : 156464, page : 3417 / 3890
code : 156464, page : 3418 / 3890
code : 156464, page : 3419 / 3890
code : 156464, page : 3420 / 3890
code : 156464, page : 3421 / 3890
code : 156464, page : 3422 / 3890
code : 156464, page : 3423 / 3890
code : 156464, page : 3424 / 3890
code : 156464, page : 3425 / 3890
code : 156464, page : 3426 / 3890
code : 156464, page : 3427 / 3890
code : 156464, page : 3428 / 3890
code : 156464, page : 3429 / 3890
code : 156464, page : 3430 / 3890
code : 156464, page : 3431 / 3890
code : 156464, page : 3432 / 3890
code : 156464, page : 3433 / 3890
code : 156464, page : 3434 / 3890
code : 156464, page : 3435 / 3890
code : 156464, page : 3436 / 3890
code : 156464, page : 3437 / 3890
code : 156464, page : 3438 / 3890
code : 156464, page : 3439 / 3890
code : 156464, page : 3440 / 3890
code : 156464, page : 3441 / 3890
code : 156464,

code : 156464, page : 3654 / 3890
code : 156464, page : 3655 / 3890
code : 156464, page : 3656 / 3890
code : 156464, page : 3657 / 3890
code : 156464, page : 3658 / 3890
code : 156464, page : 3659 / 3890
code : 156464, page : 3660 / 3890
code : 156464, page : 3661 / 3890
code : 156464, page : 3662 / 3890
code : 156464, page : 3663 / 3890
code : 156464, page : 3664 / 3890
code : 156464, page : 3665 / 3890
code : 156464, page : 3666 / 3890
code : 156464, page : 3667 / 3890
code : 156464, page : 3668 / 3890
code : 156464, page : 3669 / 3890
code : 156464, page : 3670 / 3890
code : 156464, page : 3671 / 3890
code : 156464, page : 3672 / 3890
code : 156464, page : 3673 / 3890
code : 156464, page : 3674 / 3890
code : 156464, page : 3675 / 3890
code : 156464, page : 3676 / 3890
code : 156464, page : 3677 / 3890
code : 156464, page : 3678 / 3890
code : 156464, page : 3679 / 3890
code : 156464, page : 3680 / 3890
code : 156464, page : 3681 / 3890
code : 156464, page : 3682 / 3890
code : 156464,